[View in Colaboratory](https://colab.research.google.com/github/higepon/tensorflow_seq2seq_chatbot/blob/master/seq2seq.ipynb)

### Seq2Seq Beam Search + Attention + RL
- Tensorflow 1.4.0+ is required.
- This is based on [NMT Tutorial](https://github.com/tensorflow/nmt).
- Experiment [notes](https://github.com/higepon/tensorflow_seq2seq_chatbot/wiki).



In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
import os   
import mmap 
import json
import hashlib

def colab():
    return '/tools/node/bin/forever' == os.environ['_']

def client_id():
  clients = {'dfc1d5b22ba03430800179d23e522f6f': 'client1',
                   'f8e857a2d792038820ebb2ae8d803f7c': 'client2',
                   '7628f983785173edabbde501ef8f781d': 'client3'}
  with open('/content/datalab/adc.json') as json_data:
      d = json.load(json_data)
      email =d['id_token']['email'].encode('utf-8')
      return clients[hashlib.md5(email).hexdigest()]
    
print(client_id())
current_client_id = client_id()

In [0]:
# Generate creds for the Drive FUSE library.
if not os.path.exists('drive'):
  from oauth2client.client import GoogleCredentials
  creds = GoogleCredentials.get_application_default()
  import getpass
  !echo "Auth"
  !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
  vcode = getpass.getpass()
  !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
if not os.path.exists('drive'):
  !mkdir -p drive
  !google-drive-ocamlfuse drive

drive_path = 'drive/seq2seq_data'

In [0]:
!pip -q install git+https://github.com/mrahtz/easy-tf-log#egg=easy-tf-log[tf]
from easy_tf_log import tflog, Logger
import easy_tf_log

# Hack to reset the logger state
#Logger.DEFAULT = None

from __future__ import print_function
from enum import Enum, auto

class Mode(Enum):
    Test = auto()
    TrainSeq2Seq = auto()
    TrainSeq2SeqSwapped = auto()
    TrainRL = auto()
    TweetBot = auto()

mode = Mode.Test
#mode = Mode.TrainSeq2Seq
#mode = Mode.TrainSeq2SeqSwapped
#mode = Mode.TrainRL
#mode = Mode.TweetBot


# @formatter:off
import copy as copy
from google.colab import files
import datetime
import re
import shutil
from pathlib import Path
import numpy as np
import tensorflow as tf
from tensorflow.python.layers import core as layers_core
from tensorflow.python.platform import gfile

#@formatter:on

if colab():
    import matplotlib.pyplot as plt

# Note for myself.
# You've summarized Seq2Seq
# at http://d.hatena.ne.jp/higepon/20171210/1512887715.

# If you see following error, it means your max(len(tweets of training set)) <  decoder_length.
# This should be a bug somewhere in build_decoder, but couldn't find one yet.
# You can workaround by setting hparams.decoder_length=max len of tweet in training set.
# InvalidArgumentError: logits and labels must have the same first dimension, got logits shape [48,50] and labels shape [54]
#	 [[Node: root/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT32, 

print(tf.__version__)

def data_dir():
    target_dir = "{}/chatbot_data".format(str(Path.home()))
    if not colab() and not os.path.exists(target_dir):
        raise Exception("{} not found, you may create".format(target_dir))
    return target_dir


def info(message, hparams):
    if hparams.debug_verbose:
        print(message)


def p(path):
    if colab():
        return path
    else:
        return "{}/{}".format(data_dir(), path)


def has_gpu0():
    return tf.test.gpu_device_name() == "/device:GPU:0"


class ModelDirectory(Enum):
    tweet_large = 'model/tweet_large'
    tweet_large_rl = 'model/tweet_large_rl'
    tweet_large_swapped = 'model/tweet_large_swapped'
    tweet_small = 'model/tweet_small'
    tweet_small_swapped = 'model/tweet_small_swapped'
    tweet_small_rl = 'model/tweet_small_rl'
    test_multiple1 = 'model/test_multiple1'
    test_multiple2 = 'model/test_multiple2'
    test_multiple3 = 'model/test_multiple3'
    test_distributed = 'model/test_distributed'

    @staticmethod
    def create_all_directories():
        for d in ModelDirectory:
            os.makedirs(p(d.value), exist_ok=True)


# todo
# collect all initializer
ModelDirectory.create_all_directories()  

base_hparams = tf.contrib.training.HParams(
    machine=current_client_id,
    batch_size=3,
    num_units=6,
    num_layers=2,
    vocab_size=9,
    embedding_size=8,
    learning_rate=0.01,
    learning_rate_decay=0.99,
    use_attention=False,
    encoder_length=5,
    decoder_length=5,    
    max_gradient_norm=5.0,
    beam_width=2,
    num_train_steps=100,
    debug_verbose=False,
    model_path='Please override model_directory',
    sos_id=0,
    eos_id=1,
    pad_id=2,
    unk_id=3,
    sos_token="[SOS]",
    eos_token="[EOS]",
    pad_token="[PAD]",
    unk_token="[UNK]",
)

test_hparams = copy.deepcopy(base_hparams).override_from_dict(
    {'beam_width': 0, 'num_train_steps': 100, 'learning_rate': 0.5})



test_attention_hparams = copy.deepcopy(test_hparams).override_from_dict(
    {'use_attention': True})



In [0]:
def print_hparams(hparams):
  result = {}
  for key in ['machine', 'batch_size', 'num_units', 'num_layers', 'vocab_size',
              'embedding_size', 'learning_rate', 'learning_rate_decay',
              'use_attention', 'encoder_length', 'decoder_length',
              'max_gradient_norm', 'beam_width', 'num_train_steps',
              'model_path']:
    result[key] = hparams.get(key)
  print(result)
    


In [0]:
# For debug purpose.
tf.reset_default_graph()


class ChatbotModel:
    def __init__(self, sess, hparams, model_path, scope='ChatbotModel'):
        self.sess = sess
        # todo remove
        self.hparams = hparams

        # todo
        self.model_path = model_path
        self.name = scope

        self.encoder_inputs, self.encoder_inputs_lengths, encoder_outputs, encoder_state, embedding_encoder = self._build_encoder(
            hparams, scope)
        self.decoder_inputs, self.decoder_target_lengths, self.logits = self._build_decoder(
            hparams, self.encoder_inputs_lengths, embedding_encoder,
            encoder_state, encoder_outputs)

        self.reward = tf.placeholder(tf.float32, name="reward")
        self.target_labels, self.loss, self.global_step, self.learning_rate, self.train_op = self._build_optimizer(
            hparams, self.logits)
        self.train_loss_summary = tf.summary.scalar("loss", self.loss)
        self.reward_summary = tf.summary.scalar("reward", self.reward)
        self.valiation_loss_summary = tf.summary.scalar("validation_loss", self.loss)
        self.merged_summary = tf.summary.merge_all()

        # Initialize saver after model created
        self.saver = tf.train.Saver(tf.global_variables(), max_to_keep=1)

    def restore(self):
        ckpt = tf.train.get_checkpoint_state(self.model_path)
        if ckpt:
            last_model = ckpt.model_checkpoint_path
            self.saver.restore(self.sess, last_model)
            return True
        else:
            print("Created fresh model.")
            return False

    def train(self, encoder_inputs, encoder_inputs_lengths, target_labels,
              decoder_inputs, decoder_target_lengths, reward=1.0):
        feed_dict = {
            self.encoder_inputs: encoder_inputs,
            self.encoder_inputs_lengths: encoder_inputs_lengths,
            self.target_labels: target_labels,
            self.decoder_inputs: decoder_inputs,
            self.decoder_target_lengths: decoder_target_lengths,
            # For normal Seq2Seq reward is always 1.
            self.reward: reward
        }
        _, global_step, summary, reward_summary = self.sess.run(
            [self.train_op, self.global_step, self.train_loss_summary, self.reward_summary], feed_dict=feed_dict)

        return global_step, self.learning_rate, summary, reward_summary, reward

    def batch_loss(self, encoder_inputs, encoder_inputs_lengths, target_labels,
                   decoder_inputs, decoder_target_lengths):
        feed_dict = {
            self.encoder_inputs: encoder_inputs,
            self.encoder_inputs_lengths: encoder_inputs_lengths,
            self.target_labels: target_labels,
            self.decoder_inputs: decoder_inputs,
            self.decoder_target_lengths: decoder_target_lengths,
            # For normal Seq2Seq reward is always 1.
            self.reward: 1
        }
        return self.sess.run([self.loss, self.valiation_loss_summary],
                             feed_dict=feed_dict)
      
    def input_length(self, input):
      try :
        l = input.index(self.hparams.eos_id)
        return l + 1
      except ValueError:
        return self.hparams.encoder_length

    def train_with_reward(self, infer_model, standard_seq2seq_model,
                          encoder_inputs, encoder_inputs_lengths, target_labels,
                          decoder_inputs, decoder_target_lengths,
                          dull_responses):
        infered_replies = infer_model.infer(encoder_inputs,
                                            encoder_inputs_lengths)
#        print(infered_replies)
        standard_seq2seq_encoder_inputs = []
        standard_seq2seq_encoder_inputs_lengths = []
        for reply in infered_replies:
            l = self.input_length(reply.tolist())
            standard_seq2seq_encoder_inputs_lengths.append(l)
            if l <= self.hparams.encoder_length:
                standard_seq2seq_encoder_inputs.append(np.append(reply, (
                        [self.hparams.pad_id] * (self.hparams.encoder_length - len(reply)))))
                None
            else:
                raise Exception(
                    "Inferred"
                    " reply shouldn't be longer than encoder_input")
        standard_seq2seq_encoder_inputs = np.transpose(
            np.array(standard_seq2seq_encoder_inputs))
        if True:
#           print(standard_seq2seq_encoder_inputs_lengths)
           reward1 = np.mean(standard_seq2seq_encoder_inputs_lengths) / self.hparams.encoder_length
        else:
          reward1 = standard_seq2seq_model.reward_ease_of_answering(
              standard_seq2seq_encoder_inputs,
              standard_seq2seq_encoder_inputs_lengths, dull_responses)


        reward2 = 0  # todo
        reward3 = 0  # todo
        reward = 0.25 * reward1 + 0.25 * reward2 + 0.5 * reward3
        
        return self.train(encoder_inputs, encoder_inputs_lengths, target_labels,
                          decoder_inputs, decoder_target_lengths, reward)

    def save(self, model_path=None):
        if model_path is None:
            model_path = self.model_path
        model_dir = "{}/{}".format(model_path, self.name)
        self.saver.save(self.sess, model_dir, global_step=self.global_step)

    @staticmethod
    def _softmax(x):
        return np.exp(x) / np.sum(np.exp(x), axis=0)

    def _build_optimizer(self, hparams, logits):
        # Target labels
        #   As described in doc for sparse_softmax_cross_entropy_with_logits,
        #   labels should be [batch_size, decoder_target_lengths]
        #   instead of [batch_size, decoder_target_lengths, vocab_size].
        #   So labels should have indices instead of vocab_size classes.
        target_labels = tf.placeholder(tf.int32, shape=(
            hparams.batch_size, hparams.decoder_length), name="target_labels")
        
        # Loss
        #   target_labels: [batch_size, decoder_length]
        #   logits: [batch_size, decoder_length, vocab_size]
        #   crossent: [batch_size, decoder_length]
        crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=target_labels, logits=logits)
        
        target_weights = tf.sequence_mask(self.decoder_target_lengths, hparams.decoder_length, dtype=logits.dtype)

        loss = tf.reduce_sum(
          crossent * target_weights) / tf.to_float(hparams.batch_size)
        
        # Adjust loss with reward.
        loss = tf.multiply(loss, self.reward)

        # Train
        global_step = tf.get_variable(name="global_step", shape=[],
                                      dtype=tf.int32,
                                      initializer=tf.constant_initializer(0),
                                      trainable=False)
        
        learning_rate = hparams.learning_rate # tf.train.exponential_decay(hparams.learning_rate, global_step, 100, hparams.learning_rate_decay) 

        # Calculate and clip gradients
        params = tf.trainable_variables()
        for param in params:
          info("  {}, {}, {}".format(param.name, str(param.get_shape()),
                                        param.op.device), hparams)
        
        gradients = tf.gradients(loss, params)
        clipped_gradients, _ = tf.clip_by_global_norm(
            gradients, hparams.max_gradient_norm)

        # Optimization
#        optimizer = tf.train.AdamOptimizer(learning_rate)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        device = '/cpu:0'
        if has_gpu0():
            device = '/gpu:0'
            print("!!!GPU ENABLED !!!")
        with tf.device(device):
            train_op = optimizer.apply_gradients(
                zip(clipped_gradients, params), global_step=global_step)
        return target_labels, loss, global_step, learning_rate, train_op

    @staticmethod
    def _build_encoder(hparams, scope):
        # Encoder
        #   encoder_inputs: [encoder_length, batch_size]
        #   This is time major where encoder_length comes
        #   first instead of batch_size.
        #   encoder_inputs_lengths: [batch_size]
        encoder_inputs = tf.placeholder(tf.int32, shape=(
            hparams.encoder_length, hparams.batch_size), name="encoder_inputs")
        encoder_inputs_lengths = tf.placeholder(tf.int32,
                                                shape=hparams.batch_size,
                                                name="encoder_inputs_lengtsh")

        # Embedding
        #   We originally didn't share embedding between encoder and decoder.
        #   But now we share it. It makes much easier to calculate rewards.
        #   Matrix for embedding: [vocab_size, embedding_size]
        #   Should be shared between training and inference.
        with tf.variable_scope(scope):
            embedding_encoder = tf.get_variable("embedding_encoder",
                                                [hparams.vocab_size,
                                                 hparams.embedding_size])

        # Look up embedding:
        #   encoder_inputs: [encoder_length, batch_size]
        #   encoder_emb_inputs: [encoder_length, batch_size, embedding_size]
        encoder_emb_inputs = tf.nn.embedding_lookup(embedding_encoder,
                                                    encoder_inputs)

        # LSTM cell.
        with tf.variable_scope(scope):
            # Should be shared between training and inference.
            cell_list = []
            for _ in range(hparams.num_layers):
               cell_list.append(tf.nn.rnn_cell.BasicLSTMCell(hparams.num_units))
            encoder_cell = tf.contrib.rnn.MultiRNNCell(cell_list)

        # Run Dynamic RNN
        #   encoder_outputs: [encoder_length, batch_size, num_units]
        #   encoder_state: [batch_size, num_units],
        #   this is final state of the cell for each batch.
        with tf.variable_scope(scope):
            encoder_outputs, encoder_state = tf.nn.dynamic_rnn(encoder_cell,
                                                               encoder_emb_inputs,
                                                               time_major=True,
                                                               dtype=tf.float32,
                                                               sequence_length=encoder_inputs_lengths)

        return encoder_inputs, encoder_inputs_lengths, encoder_outputs, encoder_state, embedding_encoder

    @staticmethod
    def _build_training_decoder(hparams, encoder_inputs_lengths,
                                encoder_state, encoder_outputs, decoder_cell,
                                decoder_emb_inputs, decoder_target_lengths,
                                projection_layer):
        # Decoder with helper:
        #   decoder_emb_inputs: [decoder_length, batch_size, embedding_size]
        #   decoder_target_lengths: [batch_size] vector,
        #   which represents each target sequence length.
        training_helper = tf.contrib.seq2seq.TrainingHelper(decoder_emb_inputs,
                                                            decoder_target_lengths,
                                                            time_major=True)

        # See https://github.com/tensorflow/tensorflow/issues/11904
        if hparams.use_attention:
            # Attention
            # encoder_outputs is time major, so transopse it to batch major.
            # attention_encoder_outputs: [batch_size, encoder_length, num_units]
            attention_encoder_outputs = tf.transpose(encoder_outputs, [1, 0, 2])

            # Create an attention mechanism
            attention_mechanism = tf.contrib.seq2seq.LuongAttention(
                hparams.num_units,
                attention_encoder_outputs,
                memory_sequence_length=encoder_inputs_lengths)

            wrapped_decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
                decoder_cell, attention_mechanism,
                attention_layer_size=hparams.num_units)

            initial_state = wrapped_decoder_cell.zero_state(hparams.batch_size,
                                                            tf.float32).clone(
                cell_state=encoder_state)
        else:
            wrapped_decoder_cell = decoder_cell
            initial_state = encoder_state

            # Decoder and decode
        training_decoder = tf.contrib.seq2seq.BasicDecoder(
            wrapped_decoder_cell, training_helper, initial_state,
            output_layer=projection_layer)

        # Dynamic decoding
        #   final_outputs.rnn_output: [batch_size, decoder_length,
        #                             vocab_size], list of RNN state.
        #   final_outputs.sample_id: [batch_size, decoder_length],
        #                            list of argmax of rnn_output.
        #   final_state: [batch_size, num_units],
        #                list of final state of RNN on decode process.
        #   final_sequence_lengths: [batch_size], list of each decoded sequence. 
        final_outputs, _final_state, _final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(
            training_decoder)

        if hparams.debug_verbose:
            print("rnn_output.shape=", final_outputs.rnn_output.shape)
            print("sample_id.shape=", final_outputs.sample_id.shape)
            print("final_state=", _final_state)
            print("final_sequence_lengths.shape=",
                  _final_sequence_lengths.shape)

        logits = final_outputs.rnn_output
        return logits, wrapped_decoder_cell, initial_state

    def _build_decoder(self, hparams, encoder_inputs_lengths, embedding_encoder,
                       encoder_state, encoder_outputs):
        # Decoder input
        #   decoder_inputs: [decoder_length, batch_size]
        #   decoder_target_lengths: [batch_size]
        #   This is grand truth target inputs for training.
        decoder_inputs = tf.placeholder(tf.int32, shape=(
            hparams.decoder_length, hparams.batch_size), name="decoder_inputs")
        decoder_target_lengths = tf.placeholder(tf.int32,
                                                shape=hparams.batch_size,
                                                name="decoder_target_lengths")

        # Look up embedding:
        #   decoder_inputs: [decoder_length, batch_size]
        #   decoder_emb_inp: [decoder_length, batch_size, embedding_size]
        decoder_emb_inputs = tf.nn.embedding_lookup(embedding_encoder,
                                                    decoder_inputs)

        # https://stackoverflow.com/questions/39573188/output-projection-in-seq2seq-model-tensorflow
        # Internally, a neural network operates on dense vectors of some size,
        # often 256, 512 or 1024 floats (let's say 512 for here). 
        # But at the end it needs to predict a word
        # from the vocabulary which is often much larger,
        # e.g., 40000 words. Output projection is the final linear layer
        # that converts (projects) from the internal representation
        #  to the larger one.
        # So, for example, it can consist of a 512 x 40000 parameter matrix
        # and a 40000 parameter for the bias vector.
        projection_layer = layers_core.Dense(hparams.vocab_size, use_bias=False)

        # We share this between training and inference.
        cell_list = []
        for _ in range(hparams.num_layers):
           cell_list.append(tf.nn.rnn_cell.BasicLSTMCell(hparams.num_units))
        decoder_cell = tf.contrib.rnn.MultiRNNCell(cell_list)
        
#        decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(hparams.num_units)

        # Training graph
        logits, wrapped_decoder_cell, initial_state = self._build_training_decoder(
            hparams, encoder_inputs_lengths, encoder_state, encoder_outputs,
            decoder_cell, decoder_emb_inputs, decoder_target_lengths,
            projection_layer)

        return decoder_inputs, decoder_target_lengths, logits


In [0]:
class ChatbotInferenceModel:
    def __init__(self, sess, hparams, model_path, scope='ChatbotModel'):
        self.sess = sess
        # todo remove
        self.hparams = hparams

        # todo
        self.model_path = model_path
        self.name = scope

        self.encoder_inputs, self.encoder_inputs_lengths, encoder_outputs, encoder_state, embedding_encoder = self._build_encoder(
            hparams, scope)
        self.decoder_inputs, self.decoder_target_lengths, self.replies, self.beam_replies, self.infer_logits = self._build_decoder(
            hparams, self.encoder_inputs_lengths, embedding_encoder,
            encoder_state, encoder_outputs)

        self.reward = tf.placeholder(tf.float32, name="reward")

        # we can't use variable length here, 
        # because tiled_batch requires constant length.
        self.batch_size = 1

        # Initialize saver after model created
        self.saver = tf.train.Saver(tf.global_variables(), max_to_keep=1)

    def restore(self):
        ckpt = tf.train.get_checkpoint_state(self.model_path)
        if ckpt:
            last_model = ckpt.model_checkpoint_path
            self.saver.restore(self.sess, last_model)
            return True
        else:
            print("Created fresh model.")
            return False

    def infer(self, encoder_inputs, encoder_inputs_lengths):
        # Should not call this when beam search enabled.
        assert(self.hparams.beam_width == 0)
        inference_feed_dict = {
            self.encoder_inputs: encoder_inputs,
            self.encoder_inputs_lengths: encoder_inputs_lengths,
        }
        replies = self.sess.run(self.replies, feed_dict=inference_feed_dict)
        return replies

    def infer_beam_search(self, encoder_inputs, encoder_inputs_lengths):
        # Should not call this when beam search disabled.
        assert(self.hparams.beam_width > 0)      
        inference_feed_dict = {
            self.encoder_inputs: encoder_inputs,
            self.encoder_inputs_lengths: encoder_inputs_lengths,
        }
        replies = self.sess.run(self.beam_replies,
                                feed_dict=inference_feed_dict)
        return replies
     
    # imakoko
    def infer_mi(self, swapped_model, encoder_inputs, encoder_inputs_lengths):
        inference_feed_dict = {
            self.encoder_inputs: encoder_inputs,
            self.encoder_inputs_lengths: encoder_inputs_lengths,
        }
        beam_replies = self.sess.run(self.beam_replies, feed_dict=inference_feed_dict)
        # beam_replis [batch_size, length, , batch_width]
        # for now we assume encoder_inputs is batch_size = 1
        
        swapped_encoder_inputs = beam_replies[0]
        # beam_width = batch_size
        swapped_batch_size = swapped_encoder_inputs.shape[1]
        
        # beam_replies can be shorten less than decoder_output_legth, so we pad them.
        paddings = tf.constant([[0, self.hparams.encoder_length - swapped_encoder_inputs.shape[0],], [0, 0]])
        swapped_encoder_inputs = swapped_model.sess.run(tf.pad(swapped_encoder_inputs, paddings, "CONSTANT", constant_values=self.hparams.pad_id))
        swapped_encoder_inputs_lengths = np.empty(swapped_batch_size, dtype=np.int)
        for i in range(swapped_batch_size):
          swapped_encoder_inputs_lengths[i] = swapped_encoder_inputs.shape[0]
        
        return swapped_model.infer_beam_search(swapped_encoder_inputs, swapped_encoder_inputs_lengths)
        # todo make correct length
#        for repy in beam_replies:
          # logits from swapped_model for this reply
          # cals prob for in original encoder_input
      

    def log_prob(self, encoder_inputs, encoder_inputs_lengths, expected_output):
        """Return sum of log probability of given
           one specific expected_output for encoder_inputs.
    
        Args:
            encoder_inputs: [encoder_length, batch_size], eg) tweets
            expected_output: [1, decoder_length or less than decoder_length],
            eg) One reply.
    
        Returns:
            Return log probablity of expected output for given encoder inputs.
            eg) sum of log probability of reply "Good" when given ["How are you?",
             "What's up?"]
        """
        inference_feed_dict = {
            self.encoder_inputs: encoder_inputs,
            self.encoder_inputs_lengths: encoder_inputs_lengths
        }

        # Logits
        #   logits_value: [batch_size, actual_decoder_length, vocab_size]
        logits_batch_value = self.sess.run(self.infer_logits,
                                           feed_dict=inference_feed_dict)

        sum_p = []
        # For each batch: [actual_decoder_length, vocab_size]
        for logits in logits_batch_value:
            p = 1
            # Note that expected_output and logits don't always have
            # same length, but zip takes care of the case.
            for word_id, logit in zip(expected_output, logits):
                # Apply softmax first, see definition of softmax.
                norm = (self._softmax(logit))[word_id]
                p *= norm
            p = np.log(p)
            sum_p.append(p)
        ret = np.sum(sum_p) / len(sum_p)
        return ret

    def reward_ease_of_answering(self, encoder_inputs, encoder_inputs_lengths,
                                 expected_outputs):
        """ Return reward for ease of answering. 
            See Deep Reinforcement Learning for Dialogue Generation
            for more details.
    
        Args:
            encoder_inputs: [encoder_length, batch_size], eg) tweets
            expected_outputs: [number of pre-defined dull responses,
            decoder_length or less than decoder_length].
            eg) [["I'm", "Good"], ["fine"]]
    
        Returns:
            Return reward for ease of answering.
            Note that this can be calculated
            by calling log_prob function for each dull response,
            but this function is more efficient
            because this calculated the reward at once.
        """
        inference_feed_dict = {
            self.encoder_inputs: encoder_inputs,
            self.encoder_inputs_lengths: encoder_inputs_lengths
        }

        # Logits
        #   logits_value: [batch_size, actual_decoder_length, vocab_size]
        logits_batch_value = self.sess.run(self.infer_logits,
                                           feed_dict=inference_feed_dict)

        batch_sum_p = []
        # For each batch: [actual_decoder_length, vocab_size]
        for logits in logits_batch_value:
            sum_p = []
            for expected_output in expected_outputs:
                p = 1
                # Note that expected_output and logits don't
                # always have same length, but zip takes care of the case.
                for word_id, logit in zip(expected_output, logits):
                    # Apply softmax first, see definition of softmax.
                    norm = (self._softmax(logit))[word_id]
                    p *= norm
                p = np.log(p) / len(expected_output)
                sum_p.append(p)
            one_batch_p = np.sum(sum_p)
            batch_sum_p.append(one_batch_p)
        ret = np.sum(batch_sum_p) / len(batch_sum_p)
        return -ret

    @staticmethod
    def _softmax(x):
        return np.exp(x) / np.sum(np.exp(x), axis=0)

    @staticmethod
    def _build_encoder(hparams, scope):
        # Encoder
        #   encoder_inputs: [encoder_length, batch_size]
        #   This is time major where encoder_length
        #   comes first instead of batch_size.
        #   encoder_inputs_lengths: [batch_size]
        encoder_inputs = tf.placeholder(tf.int32,
                                        shape=[hparams.encoder_length, None],
                                        name="encoder_inputs")
        encoder_inputs_lengths = tf.placeholder(tf.int32, shape=[None],
                                                name="encoder_inputs_lengths")

        # Embedding
        #   We originally didn't share embedding between encoder and decoder.
        #   But now we share it. It makes much easier to calculate rewards.
        #   Matrix for embedding: [vocab_size, embedding_size]
        #   Should be shared between training and inference.
        with tf.variable_scope(scope):
            embedding_encoder = tf.get_variable("embedding_encoder",
                                                [hparams.vocab_size,
                                                 hparams.embedding_size])

        # Look up embedding:
        #   encoder_inputs: [encoder_length, batch_size]
        #   encoder_emb_inputs: [encoder_length, batch_size, embedding_size]
        encoder_emb_inputs = tf.nn.embedding_lookup(embedding_encoder,
                                                    encoder_inputs)

        # LSTM cell.
        with tf.variable_scope(scope):
            # Should be shared between training and inference.
#            encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(hparams.num_units)
            cell_list = []
            for _ in range(hparams.num_layers):
               cell_list.append(tf.nn.rnn_cell.BasicLSTMCell(hparams.num_units))
            encoder_cell = tf.contrib.rnn.MultiRNNCell(cell_list)

        # Run Dynamic RNN
        #   encoder_outputs: [encoder_length, batch_size, num_units]
        #   encoder_state: [batch_size, num_units],
        #   this is final state of the cell for each batch.
        with tf.variable_scope(scope):
            encoder_outputs, encoder_state = tf.nn.dynamic_rnn(encoder_cell,
                                                               encoder_emb_inputs,
                                                               time_major=True,
                                                               dtype=tf.float32,
                                                               sequence_length=encoder_inputs_lengths)

        return encoder_inputs, encoder_inputs_lengths, encoder_outputs, encoder_state, embedding_encoder

    @staticmethod
    def _build_greedy_inference(hparams, embedding_encoder, encoder_state,
                                encoder_inputs_lengths, encoder_outputs,
                                decoder_cell, projection_layer):
        if hparams.beam_width > 0:
          return None, None
        
        # Greedy decoder
        dynamic_batch_size = tf.shape(encoder_inputs_lengths)[0]
        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
            embedding_encoder,
            tf.fill([dynamic_batch_size], hparams.sos_id), hparams.eos_id)

        # See https://github.com/tensorflow/tensorflow/issues/11904
        if hparams.use_attention:
            # Attention
            # encoder_outputs is time major, so transopse it to batch major.
            # attention_encoder_outputs: [batch_size, encoder_length, num_units]
            attention_encoder_outputs = tf.transpose(encoder_outputs, [1, 0, 2])

            # Create an attention mechanism
            attention_mechanism = tf.contrib.seq2seq.LuongAttention(
                hparams.num_units,
                attention_encoder_outputs,
                memory_sequence_length=encoder_inputs_lengths)

            wrapped_decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
                decoder_cell, attention_mechanism,
                attention_layer_size=hparams.num_units)

            initial_state = wrapped_decoder_cell.zero_state(dynamic_batch_size,
                                                            tf.float32).clone(
                cell_state=encoder_state)
        else:
            wrapped_decoder_cell = decoder_cell
            initial_state = encoder_state

        inference_decoder = tf.contrib.seq2seq.BasicDecoder(
            wrapped_decoder_cell, inference_helper, initial_state,
            output_layer=projection_layer)

        # len(inferred_reply) is lte encoder_length,
        # because we are targeting tweet (140 for each tweet)
        # Also by doing this,
        # we can pass the reply to other seq2seq w/o shorten it.
        maximum_iterations = hparams.encoder_length

        # Dynamic decoding
        outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
            inference_decoder, maximum_iterations=maximum_iterations)
        replies = outputs.sample_id

        # We use infer_logits instead of logits when calculating log_prob,
        # because infer_logits doesn't require decoder_target_lengths input.
        infer_logits = outputs.rnn_output
        return infer_logits, replies

    @staticmethod
    def _build_beam_search_inference(hparams, encoder_inputs_lengths,
                                     embedding_encoder, encoder_state,
                                     encoder_outputs, decoder_cell,
                                     projection_layer):
      
        if hparams.beam_width == 0:
          return None
      
        dynamic_batch_size = tf.shape(encoder_inputs_lengths)[0]
        # https://github.com/tensorflow/tensorflow/issues/11904
        if hparams.use_attention:
            # Attention
            # encoder_outputs is time major, so transopse it to batch major.
            # attention_encoder_outputs: [batch_size, encoder_length, num_units]
            attention_encoder_outputs = tf.transpose(encoder_outputs, [1, 0, 2])

            tiled_encoder_outputs = tf.contrib.seq2seq.tile_batch(
                attention_encoder_outputs, multiplier=hparams.beam_width)
            tiled_encoder_final_state = tf.contrib.seq2seq.tile_batch(
                encoder_state, multiplier=hparams.beam_width)
            tiled_encoder_inputs_lengths = tf.contrib.seq2seq.tile_batch(
                encoder_inputs_lengths, multiplier=hparams.beam_width)

            # Create an attention mechanism
            attention_mechanism = tf.contrib.seq2seq.LuongAttention(
                hparams.num_units, tiled_encoder_outputs,
                memory_sequence_length=tiled_encoder_inputs_lengths)

            wrapped_decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
                decoder_cell, attention_mechanism,
                attention_layer_size=hparams.num_units)

            decoder_initial_state = wrapped_decoder_cell.zero_state(
                dtype=tf.float32,
                batch_size=dynamic_batch_size * hparams.beam_width)
            decoder_initial_state = decoder_initial_state.clone(
                cell_state=tiled_encoder_final_state)
        else:
            wrapped_decoder_cell = decoder_cell
            decoder_initial_state = tf.contrib.seq2seq.tile_batch(encoder_state,
                                                                  multiplier=hparams.beam_width)

        # len(inferred_reply) is lte encoder_length,
        # because we are targeting tweet (140 for each tweet)
        # Also by doing this,
        # we can pass the reply to other seq2seq w/o shorten it.
        maximum_iterations = hparams.encoder_length

        inference_decoder = tf.contrib.seq2seq.BeamSearchDecoder(
            cell=wrapped_decoder_cell,
            embedding=embedding_encoder,
            start_tokens=tf.fill([dynamic_batch_size], hparams.sos_id),
            end_token=hparams.eos_id,
            initial_state=decoder_initial_state,
            beam_width=hparams.beam_width,
            output_layer=projection_layer,
            length_penalty_weight=0.0)

        # Dynamic decoding
        beam_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
            inference_decoder, maximum_iterations=maximum_iterations)
        beam_replies = beam_outputs.predicted_ids
        return beam_replies

    def _build_decoder(self, hparams, encoder_inputs_lengths, embedding_encoder,
                       encoder_state, encoder_outputs):
        # Decoder input
        #   decoder_inputs: [decoder_length, batch_size]
        #   decoder_target_lengths: [batch_size]
        #   This is grand truth target inputs for training.
        decoder_inputs = tf.placeholder(tf.int32,
                                        shape=[hparams.decoder_length, None],
                                        name="decoder_inputs")
        decoder_target_lengths = tf.placeholder(tf.int32, shape=[None],
                                                name="decoder_target_lengths")

        # https://stackoverflow.com/questions/39573188/output-projection-in-seq2seq-model-tensorflow
        # Internally, a neural network operates on dense vectors of some size,
        # often 256, 512 or 1024 floats (let's say 512 for here). 
        # But at the end it needs to predict a word
        # from the vocabulary which is often much larger,
        # e.g., 40000 words. Output projection is the final linear
        # layer that converts (projects) from the internal
        # representation to the larger one.
        # So, for example, it can consist of a 512 x 40000 parameter
        # matrix and a 40000 parameter for the bias vector.
        projection_layer = layers_core.Dense(hparams.vocab_size, use_bias=False)

        # We share this between training and inference.
#        decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(hparams.num_units)
        cell_list = []
        for _ in range(hparams.num_layers):
           cell_list.append(tf.nn.rnn_cell.BasicLSTMCell(hparams.num_units))
        decoder_cell = tf.contrib.rnn.MultiRNNCell(cell_list)
        

        # Greedy Inference graph
        infer_logits, replies = self._build_greedy_inference(hparams,
                                                             embedding_encoder,
                                                             encoder_state,
                                                             encoder_inputs_lengths,
                                                             encoder_outputs,
                                                             decoder_cell,
                                                             projection_layer)

        # Beam Search Inference graph
        beam_replies = self._build_beam_search_inference(hparams,
                                                         encoder_inputs_lengths,
                                                         embedding_encoder,
                                                         encoder_state,
                                                         encoder_outputs,
                                                         decoder_cell,
                                                         projection_layer)

        return decoder_inputs, decoder_target_lengths, replies, beam_replies, infer_logits


In [0]:
class InferenceHelper:
    def __init__(self, model, vocab, rev_vocab):
        self.model = model
        self.vocab = vocab
        self.rev_vocab = rev_vocab

    def inferences(self, tweet):
        encoder_inputs, encoder_inputs_lengths = self.create_inference_input(
            tweet)
        if self.model.hparams.beam_width == 0:
          replies = self.model.infer(encoder_inputs, encoder_inputs_lengths)
          ids = replies[0].tolist()
          return [self.sanitize_text(self.ids_to_words(ids))]
        else:
          beam_replies = self.model.infer_beam_search(encoder_inputs,
                                                      encoder_inputs_lengths)

          return [self.sanitize_text(self.ids_to_words(beam_replies[0][:, i])) for i in range(self.model.hparams.beam_width)]
        
    def sanitize_text(self, line):
      line = re.sub(r"\[EOS\]", " ", line)
      line = re.sub(r"\[UNK\]", "💩", line)
      return line

    def print_inferences(self, tweet):
        print(tweet)
        for i, reply in enumerate(self.inferences(tweet)):
            print("    [{}]{}".format(i, reply))

    def words_to_ids(self, words):
        ids = []
        for word in words:
            if word in self.vocab:
                ids.append(self.vocab[word])
            else:
                ids.append(self.model.hparams.unk_id)
        return ids

    def ids_to_words(self, ids):
        words = ""
        for id in ids:
            words += self.rev_vocab[id]
        return words

    def create_inference_input(self, text):
        inference_encoder_inputs = np.empty((self.model.hparams.encoder_length, 1),
                                            dtype=np.int)
        inference_encoder_inputs_lengths = np.empty(1, dtype=np.int)
        text = TrainDataGenerator.sanitize_line(text)
        tagger = MeCab.Tagger("-Owakati")
        words = tagger.parse(text).split()
        ids = self.words_to_ids(words)
        ids = ids[:self.model.hparams.encoder_length]
        len_ids = len(ids)
        ids.extend([self.model.hparams.pad_id] * (self.model.hparams.encoder_length - len(ids)))
        for i in range(1):
            inference_encoder_inputs[:, i] = np.array(ids, dtype=np.int)
            inference_encoder_inputs_lengths[i] = len_ids
        return inference_encoder_inputs, inference_encoder_inputs_lengths


In [0]:
class TrainDataGenerator:
    def __init__(self, source_path, hparams):
        self.source_path = source_path
        self.hparams = hparams
        basename, extension = os.path.splitext(self.source_path)
        self.enc_path = "{}_enc{}".format(basename, extension)
        self.dec_path = "{}_dec{}".format(basename, extension)
        self.enc_idx_path = "{}_enc_idx{}".format(basename, extension)
        self.dec_idx_path = "{}_dec_idx{}".format(basename, extension)
        self.dec_idx_eos_path = "{}_dec_idx_eos{}".format(basename, extension)
        self.dec_idx_sos_path = "{}_dec_idx_sos{}".format(basename, extension)
        self.dec_idx_len_path = "{}_dec_idx_len{}".format(basename, extension)

        self.enc_idx_padded_path = "{}_enc_idx_padded{}".format(basename,
                                                                extension)
        self.enc_idx_len_path = "{}_enc_idx_len{}".format(basename, extension)

        self.vocab_path = "{}_vocab{}".format(basename, extension)
        
        self.generated_files = [self.enc_path, self.dec_path, self.enc_idx_path, self.dec_idx_path, self.dec_idx_eos_path, self.dec_idx_sos_path, self.dec_idx_len_path, self.enc_idx_padded_path, self.vocab_path, self.enc_idx_len_path]
        self.max_vocab_size = hparams.vocab_size
        self.start_vocabs = [hparams.sos_token, hparams.eos_token, hparams.pad_token, hparams.unk_token]
        self.tagger = MeCab.Tagger("-Owakati")
        
    def remove_generated(self):
      for f in self.generated_files:
        if os.path.exists(f):
          os.remove(f)

    def generate(self):
        print("generating enc and dec files...")
        self._generate_enc_dec()
        print("generating vocab file...")
        self._generate_vocab()
        print("loading vocab...")
        vocab, _ = self._load_vocab()
        print("generating id files...")
        self._generate_id_file(self.enc_path, self.enc_idx_path, vocab)
        self._generate_id_file(self.dec_path, self.dec_idx_path, vocab)
        print("generating padded input file...")
        self._generate_enc_idx_padded(self.enc_idx_path,
                                      self.enc_idx_padded_path,
                                      self.enc_idx_len_path,
                                      self.hparams.encoder_length)
        print("generating dec eos/sos files...")
        self._generate_dec_idx_eos(self.dec_idx_path, self.dec_idx_eos_path,
                                   self.hparams.decoder_length)
        self._generate_dec_idx_sos(self.dec_idx_path, self.dec_idx_sos_path,
                                   self.dec_idx_len_path,
                                   self.hparams.decoder_length)
        print("done")
        return self._create_dataset()

    def _generate_id_file(self, source_path, dest_path, vocab):
        if gfile.Exists(dest_path):
            return
        with gfile.GFile(source_path, mode="rb") as f, gfile.GFile(dest_path,
                                                                   mode="wb") as of:
            for line in f:
                line = line.decode('utf-8')
                words = self.tagger.parse(line).split()
                ids = [vocab.get(w, self.hparams.unk_id) for w in words]
                of.write(" ".join([str(id) for id in ids]) + "\n")

    def _load_vocab(self):
        rev_vocab = []
        with gfile.GFile(self.vocab_path, mode="r") as f:
            rev_vocab.extend(f.readlines())
            rev_vocab = [line.strip() for line in rev_vocab]
            # Dictionary of (word, idx)
            vocab = dict([(x, y) for (y, x) in enumerate(rev_vocab)])
            return vocab, rev_vocab

    def _generate_vocab(self):
        if gfile.Exists(self.vocab_path):
            return
        vocab_dic = self._build_vocab_dic(self.enc_path)
        vocab_dic = self._build_vocab_dic(self.dec_path, vocab_dic)
        vocab_list = self.start_vocabs + sorted(vocab_dic, key=vocab_dic.get,
                                                reverse=True)
        if len(vocab_list) > self.max_vocab_size:
            vocab_list = vocab_list[:self.max_vocab_size]
        with gfile.GFile(self.vocab_path, mode="w") as vocab_file:
            for w in vocab_list:
                vocab_file.write(w + "\n")

    def _generate_enc_dec(self):
        if gfile.Exists(self.enc_path) and gfile.Exists(self.dec_path):
            return
        with gfile.GFile(self.source_path, mode="rb") as f, gfile.GFile(
                self.enc_path, mode="w+") as ef, gfile.GFile(self.dec_path,
                                                             mode="w+") as df:
            tweet = None
            reply = None
            for i, line in enumerate(f):
                line = line.decode('utf-8')
                line = self.sanitize_line(line)
                if i % 2 == 0:
                  tweet = line
                else:
                  reply = line
                  if tweet and reply:
                    ef.write(tweet)
                    df.write(reply)
                  tweet = None
                  reply = None

    def _generate_enc_idx_padded(self, source_path, dest_path, dest_len_path,
                                 max_line_len):
        if gfile.Exists(dest_path):
            return
        with open(source_path) as fin, open(dest_path,
                                            "w") as fout, open(dest_len_path,
                                                               "w") as flen:
            line = fin.readline()
            while line:
                ids = [int(x) for x in line.split()]
                if len(ids) > max_line_len:
#                    ids = ids[:max_line_len]
                    ids = ids[-max_line_len:]
                flen.write(str(len(ids)))
                flen.write("\n")
                if len(ids) < max_line_len:
                    ids.extend([self.hparams.pad_id] * (max_line_len - len(ids)))
                ids = [str(x) for x in ids]
                fout.write(" ".join(ids))
                fout.write("\n")
                line = fin.readline()

    # read decoder_idx file and append eos at the end of idx list.
    def _generate_dec_idx_eos(self, source_path, dest_path, max_line_len):
        if gfile.Exists(dest_path):
            return
        with open(source_path) as fin, open(dest_path, "w") as fout:
            line = fin.readline()
            while line:
                ids = [int(x) for x in line.split()]
                if len(ids) > max_line_len - 1:
#                    ids = ids[:max_line_len - 1]
                  ids = ids[-(max_line_len - 1):]
                ids.append(self.hparams.eos_id)
                if len(ids) < max_line_len:
                    ids.extend([self.hparams.pad_id] * (max_line_len - len(ids)))
                ids = [str(x) for x in ids]
                fout.write(" ".join(ids))
                fout.write("\n")
                line = fin.readline()

    # read decoder_idx file and put sos at the beginning of the idx list.
    # also write out length of index list.
    def _generate_dec_idx_sos(self, source_path, dest_path, dest_len_path,
                              max_line_len):
        if gfile.Exists(dest_path):
            return
        with open(source_path) as fin, open(dest_path, "w") as fout, open(
                dest_len_path, "w") as flen:
            line = fin.readline()
            while line:
                ids = [self.hparams.sos_id]
                ids.extend([int(x) for x in line.split()])
                if len(ids) > max_line_len:
                    ids = ids[:max_line_len]
                flen.write(str(len(ids)))
                flen.write("\n")
                if len(ids) < max_line_len:
                    ids.extend([self.hparams.pad_id] * (max_line_len - len(ids)))
                ids = [str(x) for x in ids]
                fout.write(" ".join(ids))
                fout.write("\n")
                line = fin.readline()

    @staticmethod
    def sanitize_line(line):
        # replace @username
        # replacing @username had bad impace where USERNAME token shows up everywhere.
#        line = re.sub(r"@([A-Za-z0-9_]+)", "USERNAME", line)
        line = re.sub(r"@([A-Za-z0-9_]+)", "", line)
        # Remove URL
        line = re.sub(r'https?:\/\/.*', "", line)
        line = line.lstrip()
        return line

    @staticmethod
    def generate_source_target_swapped(source_path):
        basename, extension = os.path.splitext(source_path)
        dest_path = "{}_swapped{}".format(basename, extension)
        with gfile.GFile(source_path, mode="rb") as fin, gfile.GFile(dest_path,
                                                                     mode="w+") as fout:
            temp = None
            for i, line in enumerate(fin):
                if i % 2 == 0:
                    temp = line
                else:
                    fout.write(line)
                    fout.write(temp)
                    temp = None
        return dest_path

    def _build_vocab_dic(self, source_path, vocab_dic={}):
        with gfile.GFile(source_path, mode="r") as f:
            for line in f:
                words = self.tagger.parse(line).split()
                for word in words:
                    if word in vocab_dic:
                        vocab_dic[word] += 1
                    else:
                        vocab_dic[word] = 1
            return vocab_dic

    @staticmethod
    def _read_file(source_path):
        f = open(source_path)
        data = f.read()
        f.close()
        return data

    def _read_vocab(self, source_path):
        rev_vocab = []
        rev_vocab.extend(self._read_file(source_path).splitlines())
        rev_vocab = [line.strip() for line in rev_vocab]
        vocab = dict([(x, y) for (y, x) in enumerate(rev_vocab)])
        return vocab, rev_vocab

    def text_line_split_dataset(self, filename):
        return tf.data.TextLineDataset(filename).map(self.split_to_int_values)

    @staticmethod
    def split_to_int_values(x):
        return tf.string_to_number(tf.string_split([x]).values, tf.int32)

    def _create_dataset(self):

        tweets_dataset = self.text_line_split_dataset(self.enc_idx_padded_path)
        tweets_lengths_dataset = tf.data.TextLineDataset(
            self.enc_idx_len_path)

        replies_sos_dataset = self.text_line_split_dataset(
            self.dec_idx_sos_path)
        replies_eos_dataset = self.text_line_split_dataset(
            self.dec_idx_eos_path)
        replies_sos_lengths_dataset = tf.data.TextLineDataset(
            self.dec_idx_len_path)

        tweets_transposed = tweets_dataset.apply(
            tf.contrib.data.batch_and_drop_remainder(
                self.hparams.batch_size)).map(
            lambda x: tf.transpose(x))
        tweets_lengths = tweets_lengths_dataset.apply(
            tf.contrib.data.batch_and_drop_remainder(self.hparams.batch_size))

        replies_with_eos_suffix = replies_eos_dataset.apply(
            tf.contrib.data.batch_and_drop_remainder(self.hparams.batch_size))
        replies_with_sos_prefix = replies_sos_dataset.apply(
            tf.contrib.data.batch_and_drop_remainder(
                self.hparams.batch_size)).map(
            lambda x: tf.transpose(x))
        replies_with_sos_suffix_lengths = replies_sos_lengths_dataset.apply(
            tf.contrib.data.batch_and_drop_remainder(
                self.hparams.batch_size))
        vocab, rev_vocab = self._read_vocab(self.vocab_path)
        return tf.data.Dataset.zip((tweets_transposed, tweets_lengths,
                                    replies_with_eos_suffix,
                                    replies_with_sos_prefix,
                                    replies_with_sos_suffix_lengths)), vocab, rev_vocab

In [0]:
class TrainDataSource:
  def __init__(self, source_path, hparams):
    download_file_if_necessary(source_path)
    generator = TrainDataGenerator(source_path=source_path,
                                   hparams=hparams)
    train_dataset, vocab, rev_vocab = generator.generate()
    self.train_dataset = train_dataset.repeat().shuffle(1024)
    # todo(higepon): Use actual validation dataset.
    self.valid_dataset = train_dataset.repeat().shuffle(1024,
                                                        seed=1234)
    self.vocab = vocab
    self.rev_vocab = rev_vocab


class Trainer:
  def __init__(self):
    self.loss_step = []
    self.val_losses = []
    self.reward_step = []
    self.reward_average = []
    self.last_saved_time = datetime.datetime.now()
    self.last_stats_time = datetime.datetime.now()
    self.num_stats_per = 20
    self.reward_history = []

  def train_rl_tweets(self, src_hparams, dst_hparams, source_path, tweets,
                      dull_responses):
    print("===== Train RL {} ====".format(source_path))
    print("src:")
    print_hparams(src_hparams)
    print("dst")
    print_hparams(dst_hparams)
    data_source = TrainDataSource(source_path, dst_hparams)

    self._rl_train_loop(data_source, src_hparams, dst_hparams, tweets,
                        dull_responses)

  def train_seq2seq_swapped(self, hparams, tweets_path, validation_tweets,
                            should_clean_saved_model=True):
    download_file_if_necessary(tweets_path)
    swapped_path = TrainDataGenerator.generate_source_target_swapped(
      tweets_path)
    self.train_seq2seq(hparams, swapped_path, validation_tweets,
                       should_clean_saved_model)

  def train_seq2seq(self, hparams, tweets_path, val_tweets,
                    should_clean_saved_model=True):
    print("===== Train Seq2Seq {} ====".format(tweets_path))
    print_hparams(hparams)
    
    if should_clean_saved_model:
      clean_model_path(hparams.model_path)
    data_source = TrainDataSource(tweets_path, hparams)
    self._train_loop(data_source, hparams, val_tweets)

  def _print_inferences(self, global_step, tweets, helper, beam_helper=None):
    print("==== {} ====".format(global_step))
    len_array = []
    for tweet in tweets:
      len_array.append(len(helper.inferences(tweet)[0]))
      helper.print_inferences(tweet)
      if beam_helper is not None:
        beam_helper.print_inferences(tweet)
    self._print_log('avg_reply_len', np.mean(len_array))

  # should this be public/private?
  @staticmethod
  def create_models(hparams):

    # See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
    config = tf.ConfigProto(log_device_placement=True)
    config.gpu_options.allow_growth = True

    train_graph = tf.Graph()
    train_sess = tf.Session(graph=train_graph, config=config)
    with train_graph.as_default():
      with tf.variable_scope('root'):
        model = ChatbotModel(train_sess, hparams,
                             model_path=hparams.model_path)
        if not model.restore():
          train_sess.run(tf.global_variables_initializer())

    # note that infer_model is not sharing variable with training model.
    infer_graph = tf.Graph()
    infer_sess = tf.Session(graph=infer_graph, config=config)
    with infer_graph.as_default():
      with tf.variable_scope('root'):
        infer_model = ChatbotInferenceModel(infer_sess, hparams,
                                            model_path=hparams.model_path)

    return model, infer_model

  # should thi be public/private
  def create_rl_models(self, dst_hparams, src_hparams):
    model, infer_model = self.create_models(
      dst_hparams)
    rl_beam_hparams = copy.deepcopy(dst_hparams).override_from_dict(
      {'beam_width': 2})
    infer_sess = infer_model.sess
    with infer_sess.graph.as_default():
      with tf.variable_scope('root', reuse=True):
        infer_beam_model = ChatbotInferenceModel(infer_sess,
                                                 rl_beam_hparams,
                                                 model_path=rl_beam_hparams.model_path)
    restored = infer_model.restore()
    if not restored:
      with rl_infer_graph.as_default():
        infer_sess.run(tf.global_variables_initializer())
    seq2seq_graph = tf.Graph()
    seq2seq_sess = tf.Session(graph=seq2seq_graph)
    with seq2seq_graph.as_default():
      with tf.variable_scope('root'):
        seq2seq_infer_model = ChatbotInferenceModel(seq2seq_sess,
                                                    src_hparams,
                                                    model_path=src_hparams.model_path)
        restored = seq2seq_infer_model.restore()
        assert restored
    return infer_beam_model, infer_model, model, seq2seq_infer_model

  def _train_loop(self, data_source,
                  hparams, tweets):
    download_model_data_if_necessary(hparams.model_path)

    device = self._available_device()
    with tf.device(device):
      model, infer_model = self.create_models(hparams)

    def my_train(**kwargs):
      data = kwargs['train_data']
      return model.train(data[0], data[1], data[2], data[3], data[4])

    self._generic_loop(data_source, hparams, infer_model, None,
                       model,
                       tweets, my_train)

  def _rl_train_loop(self, data_source, src_hparams, dst_hparams, tweets,
                     dull_responses):
    download_model_data_if_necessary(src_hparams.model_path)
    download_model_data_if_necessary(dst_hparams.model_path)

    device = self._available_device()
    with tf.device(device):
      infer_beam_model, infer_model, model, seq2seq_infer_model = self.create_rl_models(
        dst_hparams, src_hparams)

    def my_train(**kwargs):
      train_data = kwargs['train_data']
      dull_responses_ids = kwargs['dull_responses_ids']
      return model.train_with_reward(
        infer_model,
        seq2seq_infer_model,
        train_data[0],
        train_data[1],
        train_data[2],
        train_data[3],
        train_data[4],
        dull_responses_ids)

    self._generic_loop(data_source, dst_hparams, infer_model,
                       infer_beam_model, model,
                       tweets, my_train,
                       dull_responses)

  @staticmethod
  def _available_device():
    device = '/cpu:0'
    if has_gpu0():
      device = '/gpu:0'
      print("!!!GPU ENABLED !!!")
    return device

  @staticmethod
  def tokenize(infer_helper, text):
    tagger = MeCab.Tagger("-Owakati")
    words = tagger.parse(text).split()
    return infer_helper.words_to_ids(words)

  def _generic_loop(self, data_source, hparams, infer_model,
                    infer_beam_model, model,
                    tweets, train_func,
                    dull_responses=None):
    vocab = data_source.vocab
    rev_vocab = data_source.rev_vocab
    infer_helper = InferenceHelper(infer_model, vocab, rev_vocab)
    beam_helper = None if infer_beam_model is None else InferenceHelper(
      infer_beam_model, vocab, rev_vocab)

    if dull_responses is None:
      dull_responses_ids = None
    else:
      dull_responses_ids = [self.tokenize(infer_helper, text)
                            for text in
                            dull_responses]
    graph = model.sess.graph
    with graph.as_default():
      train_data_next = data_source.train_dataset.make_one_shot_iterator().get_next()
      val_data_next = data_source.valid_dataset.make_one_shot_iterator().get_next()
      easy_tf_log.set_dir(hparams.model_path)
      writer = tf.summary.FileWriter(hparams.model_path, graph)
      self.last_saved_time = datetime.datetime.now()
      for i in range(hparams.num_train_steps):
        train_data = model.sess.run(train_data_next)

        step, learning_rate, summary, reward_summary, reward = train_func(
          train_data=train_data, dull_responses_ids=dull_responses_ids)
        writer.add_summary(summary, step)
        writer.add_summary(reward_summary, step)
        self._print_reward(step, reward)

        if i != 0 and i % self.num_stats_per == 0:
          model.save(hparams.model_path)
          is_restored = infer_model.restore()
          assert is_restored
          self._print_inferences(step, tweets, infer_helper, beam_helper)
          self._compute_val_loss(step, model, val_data_next, writer)
          self._print_stats(hparams, learning_rate)
          self._plot_if_necessary()
          self._save_model_in_drive(hparams)
        else:
          print('.', end='')

  def _plot_if_necessary(self):
    if len(self.reward_average) > 0 and len(self.reward_average) % 5 == 0:
      self._plot(self.reward_step, self.reward_average,
                 y_label='reward average')
      self._plot(self.loss_step, self.val_losses, y_label='validation_loss')

  def _print_stats(self, hparams, learning_rate):
    print("learning rate", learning_rate)
    delta = (
                datetime.datetime.now() - self.last_stats_time).total_seconds() * 1000
    self._print_log("msec/data",
                    delta / hparams.batch_size / self.num_stats_per)
    self.last_stats_time = datetime.datetime.now()

  def _save_model_in_drive(self, hparams):
    now = datetime.datetime.now()
    delta_in_min = (now - self.last_saved_time).total_seconds() / 60

    if delta_in_min >= 45 :
      self.last_saved_time = datetime.datetime.now()
      save_model_in_drive(hparams.model_path)

  @staticmethod
  def _print_log(key, value):
    tflog(key, value)
    print("{}={}".format(key, round(value, 1)))

  def _print_reward(self, global_step, reward):
    tflog("reward", reward)
    self.reward_history.append(reward)
    if len(self.reward_history) == 20:
      reward_average = np.mean(self.reward_history)
      self._print_log("reward average", reward_average)
      self.reward_step.append(global_step)
      self.reward_average.append(reward_average)
      self.reward_history = []

  @staticmethod
  def _plot(x, y, x_label="step", y_label='y'):
    title = "{}_{}".format(current_client_id, y_label)
    if colab():
      plt.plot(x, y, label=title)
      plt.plot()
      plt.ylabel(y_label)
      plt.xlabel(x_label)
      plt.legend()
      plt.show()

  def _compute_val_loss(self, global_step, model, val_data_next,
                        writer):
    val_data = model.sess.run(val_data_next)
    val_loss, val_loss_log = model.batch_loss(val_data[0],
                                              val_data[1],
                                              val_data[2],
                                              val_data[3],
                                              val_data[4])
    # np.float64 to native float
    val_loss = val_loss.item()
    writer.add_summary(val_loss_log, global_step)
    self._print_log("validation loss", val_loss)
    self.loss_step.append(global_step)
    self.val_losses.append(val_loss)
    return val_loss


In [0]:
# Helper functions to test
def make_test_training_data(hparams):
    train_encoder_inputs = np.empty(
        (hparams.encoder_length, hparams.batch_size), dtype=np.int)
    train_encoder_inputs_lengths = np.empty(hparams.batch_size, dtype=np.int)
    training_target_labels = np.empty(
        (hparams.batch_size, hparams.decoder_length), dtype=np.int)
    training_decoder_inputs = np.empty(
        (hparams.decoder_length, hparams.batch_size), dtype=np.int)

    # We keep first tweet to validate inference.
    first_tweet = None

    for i in range(hparams.batch_size):
        # Tweet
        tweet = np.random.randint(low=0, high=hparams.vocab_size,
                                  size=hparams.encoder_length)
        train_encoder_inputs[:, i] = tweet
        train_encoder_inputs_lengths[i] = len(tweet)
        # Reply
        #   Note that low = 2, as 0 and 1 are reserved.
        reply = np.random.randint(low=2, high=hparams.vocab_size,
                                  size=hparams.decoder_length - 1)

        training_target_label = np.concatenate((reply, np.array([hparams.eos_id])))
        training_target_labels[i] = training_target_label

        training_decoder_input = np.concatenate(([hparams.sos_id], reply))
        training_decoder_inputs[:, i] = training_decoder_input

        if i == 0:
            first_tweet = tweet
            info("0th tweet={}".format(tweet), hparams)
            info("0th reply_with_eos_suffix={}".format(training_target_label),
                 hparams)
            info("0th reply_with_sos_prefix={}".format(training_decoder_input),
                 hparams)

        info("Tweets", hparams)
        info(train_encoder_inputs, hparams)
        info("Replies", hparams)
        info(training_target_labels, hparams)
        info(training_decoder_inputs, hparams)
    return first_tweet, train_encoder_inputs, train_encoder_inputs_lengths, training_target_labels, training_decoder_inputs


def test_training(test_hparams, model, infer_model):
    if test_hparams.use_attention:
        print("==== training model[attention] ====")
    else:
        print("==== training model ====")
    first_tweet, train_encoder_inputs, train_encoder_inputs_lengths, training_target_labels, training_decoder_inputs = make_test_training_data(
        test_hparams)
    for i in range(test_hparams.num_train_steps):
        _ = model.train(train_encoder_inputs,
                        train_encoder_inputs_lengths,
                        training_target_labels,
                        training_decoder_inputs,
                        np.ones(test_hparams.batch_size,
                                dtype=int) * test_hparams.decoder_length)
        if i % 5 == 0 and test_hparams.debug_verbose:
            print('.', end='')

        if i % 15 == 0:
            model.save()

    inference_encoder_inputs = np.empty((test_hparams.encoder_length, 1),
                                        dtype=np.int)
    inference_encoder_inputs_lengths = np.empty(1, dtype=np.int)
    for i in range(1):
        inference_encoder_inputs[:, i] = first_tweet
        inference_encoder_inputs_lengths[i] = len(first_tweet)

    # testing 
    log_prob54 = infer_model.log_prob(inference_encoder_inputs,
                                      inference_encoder_inputs_lengths,
                                      np.array([5, 4]))
    log_prob65 = infer_model.log_prob(inference_encoder_inputs,
                                      inference_encoder_inputs_lengths,
                                      np.array([6, 5]))
    print("log_prob for 54", log_prob54)
    print("log_prob for 65", log_prob65)

    reward = infer_model.reward_ease_of_answering(inference_encoder_inputs,
                                                  inference_encoder_inputs_lengths,
                                                  np.array([[5], [6]]))
    print("reward=", reward)

    if test_hparams.debug_verbose:
        print(inference_encoder_inputs)
    replies = infer_model.infer(inference_encoder_inputs,
                                inference_encoder_inputs_lengths)
    print("Infered replies", replies[0])
    print("Expected replies", training_target_labels[0])

# is this used?
def create_train_infer_models(graph, sess, hparams, force_restore=False):
    with graph.as_default():
        with tf.variable_scope('root'):
            model = ChatbotModel(sess, hparams, model_path=hparams.model_path)
           
        with tf.variable_scope('root', reuse=True):
            infer_model = ChatbotInferenceModel(sess, hparams,
                                                model_path=hparams.model_path)
            restored = model.restore()
            if not restored:
                if force_restore:
                    raise Exception("Oops, couldn't restore")
                else:
                    sess.run(tf.global_variables_initializer())
        return model, infer_model




def test_multiple_models_training():
    first_tweet, train_encoder_inputs, train_encoder_inputs_length, training_target_labels, training_decoder_inputs = make_test_training_data(
        test_hparams)
    
    hparams1 = copy.deepcopy(test_hparams).override_from_dict({'model_path': p(ModelDirectory.test_multiple1.value)})
    hparams2 = copy.deepcopy(test_hparams).override_from_dict({'model_path': p(ModelDirectory.test_multiple2.value)})
    

    graph1 = tf.Graph()
    sess1 = tf.Session(graph=graph1)
    model, infer_model = create_train_infer_models(graph1, sess1, hparams1)
    test_training(test_hparams, model, infer_model)

    graph2 = tf.Graph()
    sess2 = tf.Session(graph=graph2)
    model2, infer_model2 = create_train_infer_models(graph2, sess2, hparams2)

    test_training(test_hparams, model2, infer_model2)
    dull_responses = [[4, 6, 6], [5, 5]]
    model2.train_with_reward(infer_model2, infer_model, train_encoder_inputs,
                             train_encoder_inputs_length,
                             training_target_labels, training_decoder_inputs,
                             np.ones((test_hparams.batch_size),
                                     dtype=int) * test_hparams.decoder_length,
                             dull_responses)

    # comment out until https://github.com/tensorflow/tensorflow/issues/10731 is fixed
    graph3 = tf.Graph()
    sess3 = tf.Session(graph=graph3)
#    hparams3 = copy.deepcopy(test_hparams).override_from_dict({'model_path': p(ModelDirectory.test_multiple3.value), 'use_attention': True})
#    model3, infer_model3 = create_train_infer_models(graph3, sess3, hparams3)    
#    test_training(test_attention_hparams, model3, infer_model3)        


def test_save_restore_multiple_models_training():
  
    for d in [ModelDirectory.test_multiple1, ModelDirectory.test_multiple2, ModelDirectory.test_multiple3]:
      shutil.rmtree(p(d.value))
      os.makedirs(p(d.value), exist_ok=True)

    # Fresh model
    test_multiple_models_training()

    # Saved model
    test_multiple_models_training()


def test_distributed_pattern(hparams):
  
    for d in [hparams.model_path]:
      shutil.rmtree(p(d), ignore_errors=True)
      os.makedirs(p(d), exist_ok=True)

    print('==== test_distributed_pattern[{} {}] ===='.format('attention' if hparams.use_attention else '', 'beam' if hparams.beam_width > 0 else ''))
        
    first_tweet, train_encoder_inputs, train_encoder_inputs_lengths, training_target_labels, training_decoder_inputs = make_test_training_data(
        hparams)

    train_graph = tf.Graph()
    infer_graph = tf.Graph()
    train_sess = tf.Session(graph=train_graph)
    infer_sess = tf.Session(graph=infer_graph)
    

    model, infer_model = Trainer().create_models(hparams)

    for i in range(hparams.num_train_steps):
        _ = model.train(train_encoder_inputs,
                        train_encoder_inputs_lengths,
                        training_target_labels,
                        training_decoder_inputs,
                        np.ones(hparams.batch_size,
                                dtype=int) * hparams.decoder_length)

    model.save()

    inference_encoder_inputs = np.empty((hparams.encoder_length, 1),
                                        dtype=np.int)
    inference_encoder_inputs_lengths = np.empty(1, dtype=np.int)

    inference_encoder_inputs[:, 0] = first_tweet
    inference_encoder_inputs_lengths[0] = len(first_tweet)

    infer_model.restore()
    if hparams.beam_width == 0:
      replies = infer_model.infer(inference_encoder_inputs,
                                  inference_encoder_inputs_lengths)
      print("Inferred replies", replies[0])

    if hparams.beam_width > 0:
      beam_replies = infer_model.infer_beam_search(inference_encoder_inputs,
                                                   inference_encoder_inputs_lengths)
      print("Inferred replies candidate0", beam_replies[0][:, 0])
      print("Inferred replies candidate1", beam_replies[0][:, 1])
    print("Expected replies", training_target_labels[0])



In [0]:
if mode == Mode.Test:
  test_save_restore_multiple_models_training()

def test_distributed_one(enable_beam, enable_attention):
  hparams = copy.deepcopy(test_hparams).override_from_dict({
      'model_path': p(ModelDirectory.test_distributed.value),
      'use_attention': enable_attention,
      'beam_width': 2 if enable_beam else 0
  })
  test_distributed_pattern(hparams)
  
if mode == Mode.Test:
  test_distributed_one(enable_beam=False, enable_attention=False)
  test_distributed_one(enable_beam=False, enable_attention=True)
  test_distributed_one(enable_beam=True, enable_attention=False)
  test_distributed_one(enable_beam=False, enable_attention=True)

In [0]:
def download_file_if_necessary(file_name):
    if os.path.exists(file_name):
        return
    print("downloading {}...".format(file_name))
    !cp $drive_path/$file_name $file_name
    print("downloaded")
  
def absoluteFilePaths(directory):
   for dirpath,_,filenames in os.walk(directory):
       for f in filenames:
           yield os.path.abspath(os.path.join(dirpath, f))
  



In [0]:
def download_model_data_if_necessary(model_path):
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    print("Downloading model files...")
    !cp $drive_path/$model_path/* $model_path/
    print("done")



In [0]:
import shutil

def list_model_file(path):
    f = open('{}/checkpoint'.format(path))
    text = f.read()
    f.close()
    print(text)
    m = re.match(r".*ChatbotModel\-(\d+)", text)
    model_name = m.group(1)
    all = ["checkpoint"]
    all.extend([x for x in os.listdir(path) if re.search(model_name, x) or re.search('events.out', x)])
    print("all=", all)
    return all
  
def save_model_in_drive(model_path):
    !mkdir -p $drive_path/$model_path
    !rm -f $drive_path/$model_path/*
    print("Saving model in Google Drive...")
    for file in list_model_file(model_path):
      print("Saving ", file)
      shutil.copy2("{}/{}".format(model_path, file), "{}/{}/{}".format(drive_path, model_path, file))
    print("done")
        

In [0]:
#@title
!apt-get -qq install -y mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8
!pip install mecab-python3

In [0]:
import MeCab


def clean_model_path(model_path):
    shutil.rmtree(model_path)
    os.makedirs(model_path)


def print_header(text):
    print("============== {} ==============".format(text))




def test_tweets_small_swapped(hparams):
  replies = ["@higepon おはようございます！", "おつかれさまー。気をつけて。", "こちらこそよろしくお願いします。"]
  trainer = Trainer()
  trainer.train_seq2seq_swapped(hparams, "tweets_small.txt", replies)

# vocab size 変えたら動かなくなった
tweet_small_hparams = copy.deepcopy(base_hparams).override_from_dict(
    {
        'batch_size': 6,  # of tweets should be dividable by batch_size
        'encoder_length': 8,
        'decoder_length': 8,
        'num_units': 256,
        'num_layers': 2,
        'vocab_size': 34,
        'embedding_size': 40,
        'beam_width': 2,  # for faster iteration, this should be 10
        'num_train_steps': 200,
        'model_path': p(ModelDirectory.tweet_small.value),
        'learning_rate': 0.05,
        'use_attention': True,
    })

tweet_small_swapped_hparams = copy.deepcopy(
    tweet_small_hparams).override_from_dict(
    {'model_path': p(ModelDirectory.tweet_small_swapped.value)})

if mode == Mode.Test:
    tweets_path = "tweets_small.txt"
    TrainDataGenerator(tweets_path, tweet_small_hparams).remove_generated()
    trainer = Trainer()
    trainer.train_seq2seq(tweet_small_hparams, tweets_path,
                         ["おはようございます。寒いですね。", "さて帰ろう。明日は早い。", "今回もよろしくです。"])
    test_tweets_small_swapped(tweet_small_swapped_hparams)

In [0]:
def test_tweets_large(hparams):
  tweets = ["さて福岡行ってきます！", "誰か飲みに行こう", "熱でてるけど、でもなんか食べなきゃーと思ってアイス買おうとしたの","今日のドラマ面白そう！","お腹すいたー", "おやすみ～", "おはようございます。寒いですね。", "さて帰ろう。明日は早い。", "今回もよろしくです。", "ばいとおわ！"]
  trainer = Trainer()
  trainer.train_seq2seq(hparams, "tweets_conversation.txt", tweets, should_clean_saved_model=False)


def test_tweets_large_swapped(hparams):
  tweets = [ "今日のドラマ面白そう！","お腹すいたー", "おやすみ～", "おはようございます。寒いですね。", "さて帰ろう。明日は早い。", "今回もよろしくです。", "ばいとおわ！"]
  trainer = Trainer()
  trainer.train_seq2seq_swapped(hparams, "tweets_large.txt", tweets, should_clean_saved_model=False)


tweet_large_hparams = copy.deepcopy(base_hparams).override_from_dict(
    {
        # In typical seq2seq chatbot
        # num_layers=3, learning_rate=0.5, batch_size=64, vocab=20000-100000, learning_rate decay is 0.99, which is taken care as default parameter in AdamOptimizer.
        'batch_size': 64,  # of tweets should be dividable by batch_size
        'encoder_length': 30,
        'decoder_length': 30,
        'num_units': 1024,
        'num_layers': 3,
        'vocab_size': 100000,
        'embedding_size': 1024,
        'beam_width': 2,  # for faster iteration, this should be 10
        'num_train_steps': 1000000,
        'model_path': p(ModelDirectory.tweet_large.value),
        'learning_rate': 0.5, # For vocab_size 50000, num_layers 3, num_units 1024, tweet_large, starting learning_rate 0.05 works well, change it t0 0.01 at perplexity 800, changed it to 0.005 at 200.
        'learning_rate_decay': 0.99,
        'use_attention': True,
        # testing new restore learning rate and no USERNAME TOKEN
    })

tweet_large_swapped_hparams = copy.deepcopy(
    tweet_large_hparams).override_from_dict(
    {
        'model_path': p(ModelDirectory.tweet_large_swapped.value)
    })

if mode == Mode.TrainSeq2Seq:
    print("train seq2seq")
    print(tweet_large_hparams)
    test_tweets_large(tweet_large_hparams)
elif mode == Mode.TrainSeq2SeqSwapped:
    print("train seq2seq swapped")
    test_tweets_large_swapped(tweet_large_swapped_hparams)

In [0]:
rl_src_hparams = copy.deepcopy(tweet_large_hparams).override_from_dict({'beam_width': 0})
rl_dst_hparams = copy.deepcopy(rl_src_hparams).override_from_dict({
    'learning_rate': 0.1,
    'beam_width': 0,
    'num_train_steps': 300000,
    'model_path': p(ModelDirectory.tweet_large_rl.value),
})

tweets = ["さて福岡行ってきます！", "誰か飲みに行こう", "熱でてるけど、でもなんか食べなきゃーと思ってアイス買おうとしたの",
          "今日のドラマ面白そう！", "お腹すいたー", "おやすみ～", "おはようございます。寒いですね。",
          "さて帰ろう。明日は早い。", "今回もよろしくです。", "ばいとおわ！"]

if mode == Mode.TrainRL:
    train_large_rl(rl_src_hparams,
                   rl_dst_hparams,
                   source_path="tweets_large.txt",
                   tweets=tweets)
    

In [0]:
def download_logs(path):
  for f in absoluteFilePaths(path):
    if re.match('.*events', f):
      files.download(f)
      
def remove_saved_model(hparams):
  !mkdir -p $hparams.model_path
  !rm $hparams.model_path/*  
  
def copy_saved_model(src_hparams, dst_hparams):
  !cp $src_hparams.model_path/* $dst_hparams.model_path/
  # rm tf.logs from source so that it wouldn't be mixed in dest tf.logs.
  !rm $dst_hparams.model_path/events*
  
  
# This is designed to have small clean RL iterations many times.
def easy_train_rl_tweets(seq2seq_hparams, rl_src_hparams, rl_dst_hparams, tweets_path, validation_tweets, dull_responses):
  
  TrainDataGenerator(tweets_path, seq2seq_hparams).remove_generated()
  
  # train normal seq2seq.
  remove_saved_model(seq2seq_hparams)
  trainer = Trainer()
  trainer.train_seq2seq(seq2seq_hparams, tweets_path, validation_tweets)
  
  # Copy trained seq2seq as initial state of rl model.
  remove_saved_model(rl_dst_hparams)
  copy_saved_model(seq2seq_hparams, rl_dst_hparams)

  trainer = Trainer()
  trainer.train_rl_tweets(rl_src_hparams,   rl_dst_hparams, tweets_path, validation_tweets, dull_responses)

In [0]:
tweet_small_hparams = copy.deepcopy(
    tweet_small_hparams).override_from_dict(
    {'num_train_steps': 120,
      'decoder_length': 10,
      'encoder_length': 10
    })

rl_src_hparams = copy.deepcopy(tweet_small_hparams).override_from_dict({'beam_width': 0})
rl_dst_hparams = copy.deepcopy(rl_src_hparams).override_from_dict({
    'learning_rate': 0.1,
    'beam_width': 0,
    'num_train_steps':1000,
    'model_path': p(ModelDirectory.tweet_small_rl.value),
})

if mode == Mode.Test:  
  dull_responses = ['これ', "それ", "ありがとうございます", "おつかれ", "それなwww",
                            "おはよ(˙-˙)",
                            "おはよ！", "おはようございます！", "おつかれさまです"]
#  easy_train_rl_tweets(tweet_small_hparams,  rl_src_hparams, rl_dst_hparams, "tweets_small.txt", [ "おやすみ～", "おはようございます。寒いですね。", "今回もよろしくです。", "ばいとおわ！"], dull_responses)


In [0]:
if mode == Mode.Test:
  download_logs(ModelDirectory.tweet_small_rl.value)
  


In [0]:
num_tweets = 10000
batch_size =64
steps = 2 *int(num_tweets/batch_size)
tweet_large_hparams = copy.deepcopy(
    tweet_large_hparams).override_from_dict(
    {'num_train_steps': steps * 20,
        'vocab_size': 5000,
        'embedding_size': 256,
        'num_units': 512,
        'num_layers': 2,     
       'batch_size': batch_size,
    })

rl_large_src_hparams = copy.deepcopy(tweet_large_hparams).override_from_dict({'beam_width': 0})
rl_large_dst_hparams = copy.deepcopy(rl_large_src_hparams).override_from_dict({
    'learning_rate': 0.1,
    'beam_width': 0,
    'num_train_steps': steps * 20,
    'model_path': p(ModelDirectory.tweet_large_rl.value),
})


if mode == Mode.Test:  
  dull_responses = ['これ', "それ", "ありがとうございます", "おつかれ", "それなwww",
                            "おはよ(˙-˙)",
                            "おはよ！", "おはようございます！", "おつかれさまです"]
  easy_train_rl_tweets(tweet_large_hparams,  rl_large_src_hparams, rl_large_dst_hparams, "tweets_medium.txt", [ "おやすみ～", "おはようございます。寒いですね。", "今回もよろしくです。", "ばいとおわ！"], dull_responses)


In [0]:
if mode == Mode.Test:
  download_logs(rl_large_src_hparams.model_path)
  download_logs(rl_large_dst_hparams.model_path)

In [0]:
# N.B: This would fail if we try to download logs in the previous cell.
# My guess is tflog is somehow locking the log file when running the cell.
#download_logs()


In [0]:
def test_mutual_information(hparams, swapped_hparams):
    graph = tf.Graph()
    swapped_graph = tf.Graph()
    config = tf.ConfigProto(log_device_placement=True)
    config.gpu_options.allow_growth = True

    download_model_data_if_necessary(p(ModelDirectory.tweet.value))
    download_model_data_if_necessary(p(ModelDirectory.tweet_swapped.value))

    with graph.as_default():
        infer_sess = tf.Session(graph=graph, config=config)
        with tf.variable_scope('root'):
            model = ChatbotInferenceModel(infer_sess, hparams,
                                          model_path=p(ModelDirectory.tweet.value))
            model.restore()

    with swapped_graph.as_default():
        swap_sess = tf.Session(graph=swapped_graph, config=config)
        with tf.variable_scope('root'):
            smodel = ChatbotInferenceModel(swap_sess, swapped_hparams,
                                           model_path=p(
                                               ModelDirectory.tweet_swapped.value))
            smodel.restore()
            helper = InferenceHelper(model, vocab, rev_vocab)
            helper.print_inferences("疲れた")

            shelper = InferenceHelper(smodel, vocab, rev_vocab)
            shelper.print_inferences("お疲れ様")


#large_beam_hparams = copy.deepcopy(large_hparams)
#large_beam_hparams.beam_width = 20
#test_mutual_information(large_beam_hparams, large_swapped_hparams)


In [0]:
!pip install tweepy pyyaml

In [0]:
import tweepy
import yaml
import random

class StreamListener(tweepy.StreamListener):
    def __init__(self, api, helper):
        self.api = api
        self.helper = helper

    def on_status(self, status):
        # done hanlde @reply only
        # done print reply
        # add model paramerer
        # direct reply
        # unk reply
        # shuffle beam search
        print("{0}: {1}".format(status.text, status.author.screen_name))

        screen_name = status.author.screen_name
        # ignore my tweets
        if screen_name == self.api.me().screen_name:
          print("Ignored my tweet")
          return True
        elif status.text.startswith("@{0}".format(self.api.me().screen_name)):
          
          replies = self.helper.inferences(status.text)
          reply = random.choice(replies)
          reply = "@" + status.author.screen_name + " " + reply
          print(reply)
          self.api.update_status(status=reply,
                                 in_reply_to_status_id=status.id)
          
          return True        

    @staticmethod
    def on_error(status_code):
        print(status_code)
        return True


def listener(hparams):
  download_model_data_if_necessary(hparams.model_path)

  rl_train_graph = tf.Graph()
  rl_infer_graph = tf.Graph()
  rl_train_sess = tf.Session(graph=rl_train_graph)
  rl_infer_sess = tf.Session(graph=rl_infer_graph)
    
  _, infer_model = create_train_infer_models_in_graphs(rl_train_graph,
                                                       rl_train_sess,
                                                       rl_infer_graph,
                                                       rl_infer_sess,
                                                       hparams)
    
  source_path = "tweets_large.txt"
  download_file_if_necessary(source_path)
  generator = TrainDataGenerator(source_path=source_path, hparams=hparams)
  _, vocab, rev_vocab = generator.generate() 
  infer_model.restore()
  helper = InferenceHelper(infer_model, vocab, rev_vocab)

  config_path = 'config.yml'
  download_file_if_necessary(config_path)
  f = open(config_path, 'rt')
  cfg = yaml.load(f)['twitter']

  consumer_key = cfg['consumer_key']
  consumer_secret = cfg['consumer_secret']
  access_token = cfg['access_token']
  access_token_secret = cfg['access_token_secret']

  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_token_secret)
  api = tweepy.API(auth)

  while True:
#    try:
      stream = tweepy.Stream(auth=api.auth,
                             listener=StreamListener(api, helper))
      print("listener starting...")
      stream.userstream()
#    except Exception as e:
 #     print(e.__doc__)

           
tweet_hparams = copy.deepcopy(rl_dst_hparams).override_from_dict({'beam_width': 50})        
if mode == Mode.TweetBot:
  listener(tweet_hparams)

In [0]:
ls